In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

# Sieci neuronowe - podstawy

Do wytrenowania sieci neuronowej potrzebne są 3 elementy:

1. architektura sieci,
2. funkcja kosztu,
3. metoda optymalizacji.

Wszystkie te elementy można łatwo zaimplementować przy użyciu jednej z bibliotek do sieci neuronowych, np. PyTorch, TensorFlow lub JAX. Istnieją też biblioteki budowane na około tych najbardziej niskopoziomowych bibliotek, np. Keras lub Sonnet dla TensorFlow albo Haiku dla JAX-a. My na zajęciach będziemy używać PyTorcha orac PyTorch-Geometric do budowania sieci grafowych.

## Architektura sieci (pojemność modelu i indukcyjne ukierunkowanie)

Sieci neuronowe mają zwykle budowę warstwową, tzn. sieć składa się z sekwencji warstw które przekształcają kolejno reprezentację wejściową. Najpopularniejszą warstwą w sieciach neuronowych jest prawdopodobnie **warstwa liniowa** zwana też **w pełni połączoną** (ang. fully-connected). Każda cecha na wejściu jest połączona z każdą cechą na wyjściu warstwy, a siła tych połączeń jest uczona w trakcie treningu sieci. Ujmując to matematycznie, możemy napisać:

$$
[x_1, \dots, x_n] \cdot \begin{bmatrix} 
w_{11} & \cdots & w_{1m} \\
\vdots & \ddots & \vdots \\
w_{n1} & \cdots & w_{nm}
\end{bmatrix} = \mathbf{x}^T W = \mathbf{y}^T =[y_1, \dots, y_m],
$$

gdzie $\mathbf{x}\in\mathbb{R}^n$ jest wejściem do warstwy, a $\mathbf{y}\in\mathbb{R}^m$ to wyjście z warstwy. $W\in\mathbb{R}^{n\times m}$ to parametry warstwy zwane **wagami**. Często dodawane jest jeszcze trenowalne przesunięcie (ang. bias) do wyniku przekształcenia, tj. $\mathbf{x}^T W + \mathbf{b}^T$. Zauważmy, że pojedyncze neurony wyjściowe spełniają równanie przypominające regresję liniową:

$$
y_k = x_1 w_{1k} + x_2 w_{2k} + \cdots + x_n w_{nk} + b_k
$$

Drugim ważnym rodzajem warstw są warstwy nieliniowe. Zwykle nie mają one parametrów i służą jedynie wprowadzeniu nieliniowości do modelu. Gdyby nie te warstwy, sieć mogła by się nauczyć tylko zależności liniowych, tj. takich jak najprostsza regresja liniowa. Ze względu na efektywność najczęściej stosowaną obecnie nieliniowością jest **ReLU** (Rectified Linear Unit), o następującym wzorze:

$$
ReLU(x) = \max(x,0) = \begin{cases} x & \text{dla } x \geq 0,\\ 0 & \text{dla } x < 0. \end{cases}
$$

Inną ważną nieliniowością jest **sigmoid**, który jest mniej efektywny (wolniejszy w obliczeniu). Sigmoid był używany również w regresji logistycznej do klasyfikacji binarnej. Zwraca on liczby w przedziale (0, 1), które mogą być traktowane jako prawdopodobienstwo klasy pozytywnej.

$$
\sigma(x) = \frac{1}{1+e^{-x}}
$$

## Funkcja kosztu (definicja problemu)

Sama architektura definiuje tylko możliwości sieci, ale nie definiuje samego zadania. Do określenia, czego sieć ma się nauczyć, potrzebna nam będzie **funkcja kosztu** (ang. loss function). Będzie to cel optymalizacji, czyli funkcja, którą będziemy minimalizować (lub rzadziej maksymalizować) poprzez zmianę parametrów (wag) sieci neuronowej.

Na przykład, dla problemu regresji standardową funkcją kosztu jest **błąd średniokwadratowy** (MSE - Mean Squared Error), który mierzy średni błąd każdej predykcji podniesiony do kwadratu:

$$
\mathcal{L}(\mathcal{X}|\Theta)=\mathrm{MSE}(\mathcal{X}|\Theta) = \frac{1}{|\mathcal{X}|} \sum_{i=1}^{|\mathcal{X}|} e_i^2 = \frac{1}{|\mathcal{X}|} \sum_{i=1}^{|\mathcal{X}|} (y_i - \hat{y}_i)^2 = \frac{1}{|\mathcal{X}|} \sum_{i=1}^{|\mathcal{X}|} (y_i - f(\mathbf{x}_i|\Theta))^2
$$ 

Będziemy tę funkcję minimalizować względem parametrów $\Theta$, czyli wag sieci neuronowej.

$$
\Theta^* = {\arg \min}_\Theta \,\, \mathcal{L}(\mathcal{X}|\Theta)
$$

## Metody optymalizacji (algorytm uczenia)

Mając zdefiniowaną architekturę i cel, trzeb jeszcze dobrać algorytm poszukujący optymalnych parametrów. W klasycznym uczeniu maszynowym często mieliśmy wyprowadzone matematycznie wzory na optymalne parametry, jednak różnorodność sieci neuronowych i ich wielowarstwowość skutecznie utrudnia znajdowanie rozwiązań analitycznych.

Z pomocą przychodzą metody optymalizacji gradientowej, które obliczają gradient funkcji kosztu względem parametrów, a następnie przesuwają te parametry w kierunku przeciwnym do gradientu (jeśli chcemy funkcję minimalizować). Jest to możliwe dzięki różniczkowalności wszystkich operacji w sieci oraz regule łańcuchowej.

Najbardziej standardową metodą optymalizacji jest **SGD** (Stochastic Gradient Descent):

$$
\Theta \leftarrow \Theta - \eta \nabla \mathcal{L}(\mathcal{X}|\Theta),
$$

gdzie $\eta$ to współczynnik uczenia (ang. learning rate), który należy dobrać odpowiednio do problemu (hiperparametr). Metoda SGD jest prawdopodobnie najbardziej podstawową, a w bibliotekach do sieci zaimplementowane są również późniejsze udoskonalenia tej techniki, np. **Momentum**, które dodatkowo akumuluje gradienty z przeszłości, aby przyspieszyć zbieżność metody i uniknąć lokalnych minimum.

$$
v \leftarrow \gamma v + \eta \nabla \mathcal{L}(\mathcal{X}|\Theta), \\
\Theta \leftarrow \Theta - v.
$$

Stała momentum $\gamma$ zwykle ustawiana jest na wartość 0.9 (powinna być mniejsza od 1, by pęd się powoli tracił).


**Zadanie 1:** Uzupełnij kod do trenowania sieci o 3 wyżej wymienione komponenty. Kod ten używa fingerprintów Morgana (ECFP) do przewidywania rozpuszczalności związku.

In [3]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader

from typing import List, Tuple

from mldd.metrics import mae, rmse, rocauc, r_squared
from mldd.data import *


def train(X_train, y_train, X_valid, y_valid):
    # hyperparameters definition
    hidden_size = 512
    epochs = 50
    batch_size = 64
    learning_rate = 0.0001
    
    # model preparation
    model = ...  # TODO
    model.train()
    
    # data preparation
    dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train.reshape(-1, 1)))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # training loop
    optimizer = ...  # TODO
    loss_fn = ...  # TODO
    for epoch in trange(1, epochs + 1, leave=False):
        for X, y in tqdm(loader, leave=False):
            model.zero_grad()
            preds = model(X)
            loss = loss_fn(preds, y)
            loss.backward()
            optimizer.step()
    return model


def predict(model, X_test, y_test):
    # hyperparameters definition
    # (but this doesn't change the training results, it's only to optimize the eval speed)
    batch_size = 64

    # data preparation
    dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test.reshape(-1, 1)))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    # evaluation loop
    preds_batches = []
    with torch.no_grad():
        for X, y in tqdm(loader):
            preds = model(X)
            preds_batches.append(preds.cpu().detach().numpy())
    preds = np.concatenate(preds_batches)
    return preds


df, fold_indices = load_esol()
featurizer = ECFPFeaturizer(y_column='measured log solubility in mols per litre')
scores = []
for train_data, valid_data, test_data in cross_validate(df, fold_indices, preprocessing_fn=featurizer):
    X_train, y_train = train_data
    X_valid, y_valid = valid_data
    X_test, y_test = test_data
        
    # training
    model = train(X_train, y_train, X_valid, y_valid)
    
    # evaluation
    predictions = predict(model, X_test, y_test)
    
    rmse_score = rmse(y_test, predictions)
    mae_score = mae(y_test, predictions)
    r2_score = r_squared(y_test, predictions)
    scores.append([rmse_score, mae_score, r2_score])
    
    break  # can be removed to get results on all folds
scores = np.array(scores)
print('RMSE, MAE, R2 = ' + \
      ', '.join(f'{mean:.2f}±{std:.3f}' for mean, std in zip(scores.mean(axis=0), scores.std(axis=0))))

# Grafy molekularne

**Przypomnienie:** W matematyce grafem nazywamy obiekt składający się ze zbioru wierzchołków połączonych krawędziami, czyli $\mathcal{G} = (V, E)$, gdzie $V = \{ v_i: i \in \{1, 2, \dots, N \} \}$ oraz $E \subseteq \{ (v_i, v_j):\, v_i,v_j \in V \}$.

Grafy molekularne są specjalnego rodzaju grafami, gdzie oprócz wierzchołków (oznaczających atomy) i krawędzi (oznaczających wiązania chemiczne) mamy dodatkową informację o rodzaju atomów, a czasem także wiązań. Możemy zatem przyjąć, że dodatkowo mamy zestaw cech atomów zapisany jako macierz $X$, gdzie $X_{ij}$ to $j$-ta cecha $i$-tego atomu. Cechami mogą być zakodowane symbole atomów w postaci one-hot (1 na pozycji odpowiadającej danemu pierwiatkowi i 0 na pozostałych pozycjach), a także liczba wodorów związanych z atomem czy liczba tzw. ciężkich sąsiadów (atomów różnych od wodoru związanych z danym atomem).

Krawędzie/wiązania mogą być zapisywane na różne sposoby. Popularnym zapisem jest macierz sąsiedztwa $A$, gdzie $A_{ij}=1$ oznacza, że wierzchołki/atomy $v_i$ oraz $v_j$ są ze sobą połączone (w przeciwnym wypadku $A_{ij}=0$). W przypadku macierzy rzadkich bardziej oszczędnym zapisem jest lista par połączonych ze sobą atomów (lista par indeksów). Ten drugi zapis używany jest też przez bibliotekę PyTorch-Geometric.

W praktyce zatem graf molekularny można zapisać dwoma macierzami: $X \in \mathbb{R}^{N \times F}$ oraz $E \in \{0, 1,\dots,N-1\}^{2 \times N}$, gdzie $N$ to liczba atomów, a $F$ to liczba cech atomów.

**Zadanie 2:** Stwórz zbiór grafów molekularnych w bibliotece PyTorch-Geometric, używając tych samych danych co w zadaniu 1.

In [68]:
def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise ValueError("input {0} not in allowable set{1}:".format(
            x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))


def one_of_k_encoding_unk(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


class GraphFeaturizer(Featurizer):
    def __call__(self, df):
        graphs = []
        labels = []
        for i, row in df.iterrows():
            y = row[self.y_column]
            smiles = row.smiles
            mol = Chem.MolFromSmiles(smiles)
            
            edges = []
            for bond in mol.GetBonds():
                edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
                edges.append([bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()])
            edges = np.array(edges)
            
            nodes = []
            for atom in mol.GetAtoms():
                results = one_of_k_encoding_unk(
                    atom.GetSymbol(),
                    [
                        'Br', 'C', 'Cl', 'F', 'H', 'I', 'N', 'O', 'P', 'S', 'Unknown'
                    ]
                ) + one_of_k_encoding(
                    atom.GetDegree(),
                    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
                ) + one_of_k_encoding_unk(
                    atom.GetImplicitValence(),
                    [0, 1, 2, 3, 4, 5, 6]
                ) + [atom.GetFormalCharge(), atom.GetNumRadicalElectrons()] + one_of_k_encoding_unk(
                    atom.GetHybridization(),
                    [
                        Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2,
                        Chem.rdchem.HybridizationType.SP3, Chem.rdchem.HybridizationType.SP3D,
                        Chem.rdchem.HybridizationType.SP3D2
                    ]
                ) + [atom.GetIsAromatic()] + one_of_k_encoding_unk(
                    atom.GetTotalNumHs(),
                    [0, 1, 2, 3, 4]
                )
                nodes.append(results)
            nodes = np.array(nodes)
            
            graphs.append((nodes, edges.T))
            labels.append(y)
        labels = np.array(labels)
        return graphs, labels

In [69]:
from torch_geometric.data import ...  # TODO


class GraphDataset(...):  # TODO
    def __init__(self, X, y, root, transform=None, pre_transform=None):
        self.dataset = (X, y)
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['data.pt']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        data = []
        ...  # TODO (data should be a list of graphs)
        torch.save(data, self.raw_paths[0])
        

    def process(self):
        # Read data into huge `Data` list.
        data_list = torch.load(self.raw_paths[0])
        
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# Grafowe sieci neuronowe

Grafowe sieci neuronowe (GNN - Graph Neural Networks) potrafią przetwarzać grafy podane na wejściu modelu. Warstwa grafowa przyjmuje na wejściu graf z reprezentacjami poszczególnych atomów i zwraca na wyjściu graf o takiej samej strukturze (nie zmienia krawędzi) ale ze zaktualizowanymi reprezentacjami w wierzchołkach. W taki sposób możemy stworzyć sieć do **klasyfikacji wierzchołków** lub użyć operacji odczytu grafu przy pomocy wybranej funkcji agregującej wierzchołki, np. średniej. Wynikiem tej operacji będzie wektor opisujący cały graf, który może nam posłużyć do **klasyfikacji grafów**.

## Message Passing Neural Networks (MPNN)

MPNN jest bardzo ogólnym opisem sieci grafowej, do którego można dopasować wiele istniejących architektur sieci. W sieci MPNN wiadomości (ang. **message**) z sąsiednich wierzchołków są przesyłane do poszczególnych wierzchołków i zachodzi operacja aktualizacji reprezentacji (ang. **update**). Operacja ta jest powtarzana kilkukrotnie, a następnie dokonuje się odczytu (ang. **readout**) całego grafu.

$$
\mathbf{m}_i^{t+1} = \sum_{j\in\mathcal{N}(i)} M_t(\mathbf{h}_i^t, \mathbf{h}_j^t, \mathbf{e}_{ij})\\
\mathbf{h}_i^{t+1} = U_t(\mathbf{h}_i^t, \mathbf{m}_i^{t+1}) \\
\hat{y} = R(\{\mathbf{h}_i^T \, | \, i \in \mathcal{G} \})
$$

## Graph Convolutional Networks (GCN)

Sieci konwolucyjne/splotowe działają na grafach podobnie jak na obrazach. Reprezentacje wszystkich wierzchołków w promieniu jednego wiązania są przemnażane przez wagi i sumowane. Dodatkowo wartości te są normalizowane uwzględniając stopień wierzchołka.

$$
\mathbf{h}_i^{t+1} = W^T \sum_{j\in\mathcal{N}(i)\cup \{i\}} \frac{e_{ij}}{\sqrt{\hat{d}_i \hat{d}_j}} \mathbf{h}_j^t
$$

## Graph Isomorphism Networks (GIN)

$$
\mathbf{h}_i^{t+1} = W^T \left( (1+\varepsilon)\mathbf{h}_i^t + \sum_{j\in\mathcal{N}(i)} \mathbf{h}_j^t \right)
$$

## GraphSAGE

$$
\mathbf{h}_i^{t+1} = W_1 \mathbf{h}_i^t + W_2 \frac{1}{|\mathcal{N}(i)|} \sum_{j\in\mathcal{N}(i)} \mathbf{h}_j^t
$$

## Graph Attention Networks (GAT)

$$
\mathbf{h}_i^{t+1} = \sum_{j\in\mathcal{N}(i)\cup \{i\}} \alpha_{ij} W \mathbf{h}_j^t,\\
\alpha_{ij} = \frac{\exp\left( LeakyReLU(\mathbf{a}[W\mathbf{h}_i^t \| W\mathbf{h}_j^t])\right)}{\sum_{k\in\mathcal{N}(i) \cup \{i\}}\exp\left( LeakyReLU(\mathbf{a}[W\mathbf{h}_i^t \| W\mathbf{h}_k^t])\right)}
$$

**Zadanie 3:** Użyj przygotowanych grafów molekularnych do przewidywania rozpuszczalności.

In [109]:
from torch_geometric.loader import DataLoader as GraphDataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, Sequential as GraphSequential


def train(X_train, y_train, X_valid, y_valid):
    # hyperparameters definition
    hidden_size = 512
    epochs = 50
    batch_size = 64
    learning_rate = 0.0001
    
    # model preparation
    model = ...  # TODO
    model.train()
    
    # data preparation
    dataset = GraphDataset(X_train, y_train.reshape(-1, 1), root='esol-train')
    loader = GraphDataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # training loop
    optimizer = ...  # TODO
    loss_fn = ...  # TODO
    for epoch in trange(1, epochs + 1, leave=False):
        for data in tqdm(loader, leave=False):
            x, edge_index, batch, y = data.x, data.edge_index, data.batch, data.y
            
            model.zero_grad()
            preds = model(x, edge_index, batch)
            loss = loss_fn(preds, y.reshape(-1, 1))
            loss.backward()
            optimizer.step()
    return model


def predict(model, X_test, y_test):
    # hyperparameters definition
    # (but this doesn't change the training results, it's only to optimize the eval speed)
    batch_size = 64

    # data preparation
    dataset = GraphDataset(X_test, y_test.reshape(-1, 1), root='esol-test')
    loader = GraphDataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    # evaluation loop
    preds_batches = []
    with torch.no_grad():
        for data in tqdm(loader):
            x, edge_index, batch = data.x, data.edge_index, data.batch
            
            preds = model(x, edge_index, batch)
            preds_batches.append(preds.cpu().detach().numpy())
    preds = np.concatenate(preds_batches)
    return preds


df, fold_indices = load_esol()
featurizer = GraphFeaturizer(y_column='measured log solubility in mols per litre')
scores = []
for train_data, valid_data, test_data in cross_validate(df, fold_indices, preprocessing_fn=featurizer):
    X_train, y_train = train_data
    X_valid, y_valid = valid_data
    X_test, y_test = test_data
            
    # training
    model = train(X_train, y_train, X_valid, y_valid)
    
    # evaluation
    predictions = predict(model, X_test, y_test)
    
    rmse_score = rmse(y_test, predictions.flatten())
    mae_score = mae(y_test, predictions.flatten())
    r2_score = r_squared(y_test, predictions.flatten())
    scores.append([rmse_score, mae_score, r2_score])
    
    break  # can be removed to get results on all folds
scores = np.array(scores)
print('RMSE, MAE, R2 = ' + \
      ', '.join(f'{mean:.2f}±{std:.3f}' for mean, std in zip(scores.mean(axis=0), scores.std(axis=0))))


RMSE, MAE, R2 = 1.15±0.000, 0.87±0.000, 0.68±0.000


# Bonus: Interpretowalność - Grad-CAM dla grafów

$$
\alpha_k^{l,c}=\frac{1}{N}\sum_{n=1}^N \frac{\partial y^c}{\partial F_{k,n}^l},\\
L^c(l,n) = ReLU\left(\sum_k \alpha_k^{l,c} F_{k,n}^l (X, A)\right)
$$

In [133]:
class GraphNeuralNetwork(torch.nn.Module):
    def __init__(self, hidden_size):
        super(GraphNeuralNetwork, self).__init__()
        ...  # TODO
    
    def activations_hook(self, grad):
        self.final_conv_grads = grad
    
    def forward(self, x, edge_index, batch):
        ...  # TODO
        with torch.enable_grad():
            self.final_conv_acts = ...  # TODO
        self.final_conv_acts.register_hook(self.activations_hook)
        ...  # TODO
        return ...  # TODO

In [5]:
from torch_geometric.loader import DataLoader as GraphDataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, Sequential as GraphSequential


def train(X_train, y_train, X_valid, y_valid):
    # hyperparameters definition
    hidden_size = 512
    epochs = 50
    batch_size = 64
    learning_rate = 0.0001
    
    # model preparation
    model = ...  # TODO
    model.train()
    
    # data preparation
    dataset = GraphDataset(X_train, y_train.reshape(-1, 1), root='esol-train')
    loader = GraphDataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # training loop
    optimizer = ...  # TODO
    loss_fn = ...  # TODO
    for epoch in trange(1, epochs + 1, leave=False):
        for data in tqdm(loader, leave=False):
            x, edge_index, batch, y = data.x, data.edge_index, data.batch, data.y
            
            model.zero_grad()
            preds = model(x, edge_index, batch)
            loss = loss_fn(preds, y.reshape(-1, 1))
            loss.backward()
            optimizer.step()
    return model


def predict(model, X_test, y_test):
    # hyperparameters definition
    # (but this doesn't change the training results, it's only to optimize the eval speed)
    batch_size = 64

    # data preparation
    dataset = GraphDataset(X_test, y_test.reshape(-1, 1), root='esol-test')
    loader = GraphDataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    # evaluation loop
    preds_batches = []
    with torch.no_grad():
        for data in tqdm(loader):
            x, edge_index, batch = data.x, data.edge_index, data.batch
            
            preds = model(x, edge_index, batch)
            preds_batches.append(preds.cpu().detach().numpy())
    preds = np.concatenate(preds_batches)
    return preds


df, fold_indices = load_esol()
featurizer = GraphFeaturizer(y_column='measured log solubility in mols per litre')
scores = []
for train_data, valid_data, test_data in cross_validate(df, fold_indices, preprocessing_fn=featurizer):
    X_train, y_train = train_data
    X_valid, y_valid = valid_data
    X_test, y_test = test_data
            
    # training
    model = train(X_train, y_train, X_valid, y_valid)
    
    # evaluation
    predictions = predict(model, X_test, y_test)
    
    rmse_score = rmse(y_test, predictions.flatten())
    mae_score = mae(y_test, predictions.flatten())
    r2_score = r_squared(y_test, predictions.flatten())
    scores.append([rmse_score, mae_score, r2_score])
    
    break  # can be removed to get results on all folds
scores = np.array(scores)
print('RMSE, MAE, R2 = ' + \
      ', '.join(f'{mean:.2f}±{std:.3f}' for mean, std in zip(scores.mean(axis=0), scores.std(axis=0))))

In [137]:
import torch.nn.functional as F


def grad_cam(final_conv_acts, final_conv_grads):
    node_heat_map = []
    alphas = ...  # TODO (formula 1)
    for n in range(final_conv_acts.shape[0]): # nth node
        node_heat = ...  # TODO (formula 2)
        node_heat_map.append(node_heat)
    return node_heat_map

In [4]:
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

import matplotlib
import matplotlib.cm as cm


compound_idx = 110
mol = Chem.MolFromSmiles(df.iloc[fold_indices[0][compound_idx]].smiles)

graph = test_data[0][compound_idx]
X, E = graph
data = Data(
    x=torch.FloatTensor(X),
    edge_index=torch.LongTensor(E)
)

x, edge_index, batch = data.x, data.edge_index, data.batch

model(x, edge_index, torch.zeros(x.shape[0], dtype=torch.int64))
atom_weights = grad_cam(model.final_conv_acts, model.final_conv_grads)

atom_weights = np.array(atom_weights)
if (atom_weights > 0.).any():
    atom_weights = atom_weights / atom_weights.max() / 2

if len(atom_weights) > 0:
    norm = matplotlib.colors.Normalize(vmin=-1, vmax=1)
    cmap = cm.get_cmap('bwr')
    plt_colors = cm.ScalarMappable(norm=norm, cmap=cmap)
    atom_colors = {
        i: plt_colors.to_rgba(atom_weights[i]) for i in range(len(atom_weights))
    }
    highlight_kwargs = {
        'highlightAtoms': list(range(len(atom_weights))),
        'highlightBonds': [],
        'highlightAtomColors': atom_colors
    }

d = rdMolDraw2D.MolDraw2DSVG(500, 500) # or MolDraw2DCairo to get PNGs
rdMolDraw2D.PrepareAndDrawMolecule(d, mol, **highlight_kwargs)
d.FinishDrawing()
svg = d.GetDrawingText()
svg = svg.replace('svg:', '')
SVG(svg)